### Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Loads .env in current directory

required_vars = [
    "OPENAI_API_KEY",
    "LANGFUSE_PUBLIC_KEY",
    "LANGFUSE_SECRET_KEY",
    "LANGFUSE_HOST",
    "POSTGRES_URL",
]

for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"Missing env var: {var}")

print("All environment variables loaded")


All environment variables loaded


### Create a Simple Knowledge Base File for RAG

In [2]:
with open("context.txt", "w") as f:
    f.write("""
Elon Musk is the CEO of Tesla and SpaceX. He founded Neuralink and co-founded OpenAI.
LangChain is a framework for building LLM-powered applications.
LangGraph enables stateful, multi-step AI workflows.
""")
print("context.txt file created")


context.txt file created


### Build the Vector Store for RAG (Chroma)

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = TextLoader("context.txt")
documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

In [ ]:
vectorstore = Chroma.from_documents(
    split_docs,
    OpenAIEmbeddings(model="text-embedding-3-small"),
    persist_directory="rag_db"
)

vectorstore.persist()

C:\Users\HirudikaAnupama\AppData\Local\Temp\ipykernel_12664\4165993874.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


### Define LLM & RAG Search Helpers

In [9]:
from langchain.chat_models import ChatOpenAI
from langfuse.langchain import CallbackHandler

def get_llm():
    cb_handler = CallbackHandler()
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, callbacks=[cb_handler])
    return llm, cb_handler

def search_docs(query):
    db = Chroma(persist_directory="rag_db", embedding_function=OpenAIEmbeddings())
    results = db.similarity_search(query)
    return results


### LangGraph Workflow Setup (Postgres + Nodes)

In [48]:
from pydantic import BaseModel
from typing import Optional
import os
import asyncio

from langgraph.graph import StateGraph
from langgraph.types import Command
from langgraph.checkpoint.postgres import PostgresSaver

class StateSchema(BaseModel):
    question: Optional[str] = None
    context: Optional[str] = None
    answer: Optional[str] = None

llm, handler = get_llm()

def decide_step(state):
    question = (state.question or "").lower()
    print("[decide_step] question:", question)
    if any(k in question for k in ["who", "what", "tell me"]):
        return Command.next("search")
    return Command.next("direct_answer")

def search_step(state):
    question = state.question
    print("[search_step] Searching docs for:", question)
    docs = search_docs(question)
    state.context = "\n".join([doc.page_content for doc in docs])
    return Command.next("generate_answer")

def generate_answer_step(state):
    prompt = f"Use this context to answer:\n\n{state.context}\n\nQ: {state.question}"
    print("[generate_answer_step] Generating answer...")
    state.answer = llm.predict(prompt)
    return Command.end()

def direct_answer_step(state):
    print("[direct_answer_step] Directly answering...")
    state.answer = llm.predict(state.question)
    return Command.end()

def build_graph():
    saver = PostgresSaver.from_conn_string(os.environ["POSTGRES_URL"])
    graph = StateGraph(state_schema=StateSchema)
    graph.add_node("decide", decide_step)
    graph.add_node("search", search_step)
    graph.add_node("generate_answer", generate_answer_step)
    graph.add_node("direct_answer", direct_answer_step)
    graph.set_entry_point("decide")
    graph.add_edge("search", "generate_answer")
    compiled = graph.compile(checkpointer=saver)
    return compiled




In [51]:
saver = PostgresSaver.from_conn_string(os.environ["POSTGRES_URL"])

graph = StateGraph(state_schema=StateSchema)
# add nodes, edges...
compiled_graph = graph.compile(checkpointer=saver)

with saver:
    result = compiled_graph.invoke(
        {"question": "Who is Elon Musk?"},
        config={
            "checkpoint_ns": "rag_agent",
            "checkpoint_id": "session1",
            "thread_id": "main_thread",
        }
    )
    print("Answer:", result.get("answer"))


ValueError: Graph must have an entrypoint: add at least one edge from START to another node